# ARIMA MODEL 
#### Split train-test-validate: 65-25-10

##### Step 1. Import modules and load dataset

In [1]:
import numpy as np
import pandas as pd
import math
import statistics as st
import matplotlib.pyplot as plt 
import seaborn as sns
from scipy.stats import skew
from scipy.stats import kurtosis

In [2]:
df = pd.read_csv('HPG Historical Data.csv',delimiter=";")
df

,Date,Price,Open,High,Low,Vol.,Change %
0,03/01/2024,30.850,31.200,31.200,30.650,26.96M,"-0,48%"
1,02/29/2024,31.000,30.750,31.150,30.500,33.58M,"1,31%"
2,02/28/2024,30.600,30.400,30.950,30.150,35.20M,"0,82%"
3,02/27/2024,30.350,28.900,30.450,28.800,86.84M,"5,38%"
4,02/26/2024,28.800,28.400,28.800,28.300,19.22M,"1,05%"
...,...,...,...,...,...,...,...
1247,03/07/2019,12.001,12.318,12.335,12.001,12.47M,"-2,57%"
1248,03/06/2019,12.318,12.318,12.388,12.107,9.24M,"-0,14%"
1249,03/05/2019,12.335,12.353,12.529,12.300,15.62M,"-0,85%"
1250,03/04/2019,12.441,12.230,12.441,12.212,13.74M,"3,21%"


In [3]:
# Convert the data types
df['Date'] = pd.to_datetime(df['Date'])

##### Step 2. Selecting attribute 

In [4]:
df = df[['Price']]
df = df.dropna() 
df = df.reset_index(drop=True) 
df

,Price
0,30.850
1,31.000
2,30.600
3,30.350
4,28.800
...,...
1247,12.001
1248,12.318
1249,12.335
1250,12.441


##### Step 3. Split the dataset into train, test, validate

In [5]:
# Calculate the number of samples for each set
train_size = int(0.65 * len(df))
test_size = int(0.25 * len(df))
val_size = len(df) - train_size - test_size

# Split data into sets
train_data = df[:train_size]
test_data = df[train_size:train_size+test_size]
val_data = df[train_size+test_size:]

# Check the size of each set
print("Train data size:", len(train_data))
print("Test data size:", len(test_data))
print("Validate data size:", len(val_data))

Train data size: 813
Test data size: 313
Validate data size: 126
